In [1]:
from pathlib import Path
import shutil
import time
from typing import Dict
import random
import hydra
from hydra.utils import get_original_cwd
from omegaconf import OmegaConf, open_dict
from torch.utils.tensorboard import SummaryWriter
from collections import deque
from tqdm import tqdm
import wandb
import pprint
# import gymnasium as gym
import gym
import numpy as np
import torch
from torch.linalg import matrix_norm
from tensordict.tensordict import TensorDict
from modules.hyper_agent import HyperAgent
from modules.utils import set_seed, make_envpool_env
from itertools import combinations
import logging
import os
import seaborn as sns
import matplotlib.pyplot as plt
from gym.spaces import Box as GymBox
from gym.spaces import Discrete as GymDiscrete
from gymnasium.spaces import Box as GymnasiumBox
from gymnasium.spaces import Discrete as GymnasiumDiscrete


/home/kukjin/anaconda3/envs/HMRL/lib/python3.9/site-packages/lightning_fabric/__init__.py:36: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('lightning_fabric')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__("pkg_resources").declare_namespace(__name__)
/home/kukjin/anaconda3/envs/HMRL/lib/python3.9/site-packages/torchmetrics/utilities/imports.py:24: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _PYTHON_LOWER_3_8 = LooseVersion(_PYTHON_VERSION) < LooseVersion("3.8")
/home/kukjin/anaconda3/envs/HMRL/lib/python3.9/site-packages/torchvision/transforms/_functional_pil.py:242: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  interpolation: int = Image.BILINEAR,
/home/k

[KeOps] Warning : Cuda libraries were not detected on the system ; using cpu only mode


In [2]:
from omegaconf import OmegaConf
cfg_path = "/home/kukjin/Projects/HMRL/configs/hyper_trainer.yaml"
nn_cfg_path = "/home/kukjin/Projects/HMRL/configs/nn/hyper_policy.yaml"
ppo_cfg_path = "/home/kukjin/Projects/HMRL/configs/ppo/ppo.yaml"
cfg = OmegaConf.load(cfg_path)
nn_cfg = OmegaConf.load(nn_cfg_path)
ppo_cfg = OmegaConf.load(ppo_cfg_path)
cfg.nn = nn_cfg
cfg.ppo = ppo_cfg


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchopt
from torch.func import grad, grad_and_value, vmap

In [4]:
env_id = "HalfCheetah-v4"

In [5]:
env = make_envpool_env(0, env_id, cfg)

Train env 1: HalfCheetah-v4 is loaded


/home/kukjin/anaconda3/envs/HMRL/lib/python3.9/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [6]:
from modules.hyper_agent import HyperAgent
device = "cuda:1"
hyperagent = HyperAgent(cfg).to(device)


In [7]:
exp_cfg = cfg.experiment

In [8]:
batch_size = int(exp_cfg.num_envs * exp_cfg.num_rollout_steps)
minibatch_size = int(batch_size // cfg.ppo.num_minibatches)
total_num_params = sum([np.prod(p.size()) for p in hyperagent.parameters()])
print(batch_size)
print(minibatch_size)
print(total_num_params)


128
5
390464


In [9]:

from modules.hyper_agent import HyperAgent
device = "cuda:1"
hyperagent = HyperAgent(cfg).to(device)

obs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_observation_space.shape).to(device)
actions = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_action_space.shape).to(device)
logprobs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
rewards = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
dones = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
values = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
storage = TensorDict({
            "obs": obs,
            "actions": actions,
            "rewards": rewards,
            "dones": dones,
            }, batch_size=[exp_cfg.num_rollout_steps, exp_cfg.num_envs])

hyper_rollout_steps = 4
# 17 + 64 + 64 + 6 + 64 + 64 + 6
obs_dim = np.prod(env.observation_space.shape)
mid_dim = 64
act_dim = np.prod(env.action_space.shape)
weight_action_dim = obs_dim + 4*mid_dim + 2*act_dim
hyper_storage = TensorDict({
                "trajectories": torch.stack([storage.clone()for i in range(hyper_rollout_steps)], dim=0),
                "weight_actions": torch.zeros((hyper_rollout_steps, weight_action_dim)).to(device),
                "logprobs": torch.zeros((hyper_rollout_steps, 1)).to(device),
                "dones": torch.zeros((hyper_rollout_steps, 1)).to(device),
                "values": torch.zeros((hyper_rollout_steps, 1)).to(device),
                }, batch_size=[hyper_rollout_steps,])

In [10]:
hyper_storage

TensorDict(
    fields={
        dones: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        logprobs: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        trajectories: LazyStackedTensorDict(
            fields={
                actions: Tensor(shape=torch.Size([4, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
                dones: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
                obs: Tensor(shape=torch.Size([4, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
                rewards: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
            batch_size=torch.Size([4, 128, 1]),
            device=None,
            is_shared=False),
        values: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        weight_actions: Tensor(sh

In [11]:
next_obs = env.reset()
next_obs = torch.Tensor(next_obs).to(device)
next_done = torch.zeros(exp_cfg.num_envs).to(device)
print(next_obs.shape)
print(next_done.shape)
actions = np.array([env.action_space.sample() for i in range(next_obs.shape[0])])
print(actions.shape)


kwargs: {}
torch.Size([1, 17])
torch.Size([1])
(1, 6)


In [12]:
action = torch.zeros_like(torch.tensor(actions))
rewards = torch.zeros([action.shape[0], ])
print(rewards.shape)
obs_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_observation_space.shape).to(device)
actions_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_action_space.shape).to(device)
rewards_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
obs_ary[0] = next_obs
actions_ary[0] = action
rewards_ary[0] = rewards
init_traj = TensorDict({
            "obs": obs_ary,
            "actions": actions_ary,
            "rewards": rewards_ary,
            }, batch_size=[exp_cfg.num_rollout_steps, exp_cfg.num_envs])



torch.Size([1])


In [13]:
print(obs_ary[0:2])

print(obs_ary[0:2].shape)
print(actions_ary[0:2].shape)
print(rewards_ary[0:2].shape)



tensor([[[ 6.6114e-04, -2.7240e-04,  9.5442e-04, -8.1753e-04, -2.0646e-04,
          -2.9157e-04, -2.6721e-05,  9.7685e-04,  5.3563e-04,  1.0997e-03,
          -5.5710e-04,  6.9071e-04, -4.0816e-04,  2.7745e-04, -2.9971e-04,
          -2.3282e-04,  6.5928e-05]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]], device='cuda:1')
torch.Size([2, 1, 17])
torch.Size([2, 1, 6])
torch.Size([2, 1])


In [14]:
init_traj[0:2]['obs']

tensor([[[ 6.6114e-04, -2.7240e-04,  9.5442e-04, -8.1753e-04, -2.0646e-04,
          -2.9157e-04, -2.6721e-05,  9.7685e-04,  5.3563e-04,  1.0997e-03,
          -5.5710e-04,  6.9071e-04, -4.0816e-04,  2.7745e-04, -2.9971e-04,
          -2.3282e-04,  6.5928e-05]],

        [[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00]]], device='cuda:1')

In [15]:
weight_action, log_prob, entropy, value, policy = hyperagent.get_action_value_policy(
    init_traj,
    env,
    64,
    None
)

In [16]:
total_num_updates = exp_cfg.total_timesteps // batch_size
start_update_idx = 1 


In [17]:
envs_returns = dict()
envs_lengths = dict()

In [18]:
hyper_storage["trajectories"][0]

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([128, 1]),
    device=None,
    is_shared=False)

In [19]:
# next_obs, reward, done, infos = env.step(action.cpu().numpy())
# traj, reward, done, infos = env.step(action.cpu().numpy())



In [20]:
hyper_storage["trajectories"][0]["rewards"].shape

torch.Size([128, 1])

In [21]:
hyper_storage["trajectories"][0]['dones']

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [32]:
def rollout(current_hyper_timesteps, env, policy, hyper_storage, 
            next_obs, hyperdone, envs_returns, envs_lengths, global_step):
    policy = policy.to(device)
    action_scale = torch.tensor((env.action_space.high - env.action_space.low) / 2.0, dtype=torch.float32).to(device)
    action_bias = torch.tensor((env.action_space.high + env.action_space.low) / 2.0, dtype=torch.float32).to(device)
    next_done = hyperdone
    for step in range(0, cfg.experiment.num_rollout_steps):
        global_step += 1 * exp_cfg.num_envs
        hyper_storage["trajectories"][current_hyper_timesteps]["obs"][step] = next_obs
        hyper_storage["trajectories"][current_hyper_timesteps]["dones"][step] = next_done

        with torch.no_grad():
            action = policy(next_obs, action_scale, action_bias)
        hyper_storage["trajectories"][current_hyper_timesteps]["actions"][step] = action

        # TRY NOT TO MODIFY: execute the game and log data.
        next_obs, reward, done, infos = env.step(action.cpu().numpy())
        hyper_storage["trajectories"][current_hyper_timesteps]["rewards"][step] = torch.tensor(reward).to(device).view(-1)
        next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(done).to(device)
        for k, d in enumerate(done):
            if d:
                print(f"global_step: {global_step}. episodic_return={infos['r'][k]}")
                print(f"global_step: {global_step}.  episodic_length={infos['l'][k]}")
                envs_returns['episodic_return'] = infos["r"][k]
                envs_lengths['episodic_length'] = infos["l"][k]
    trajectories = hyper_storage["trajectories"][current_hyper_timesteps]
    gammas = torch.tensor([cfg.ppo.gamma ** i for i in range(cfg.experiment.num_rollout_steps)]).to(device)
    reward_lst = hyper_storage["trajectories"][current_hyper_timesteps]["rewards"].view(-1)
    reward = torch.dot(gammas, reward_lst)
    hyperdone = 1.0 if 1.0 in hyper_storage["trajectories"][0]['dones'] else 0.0
    hyperdone = torch.tensor(hyperdone).to(device)
    # global_step += 1 * exp_cfg.num_envs * exp_cfg.num_rollout_steps
    
    return trajectories, reward, hyperdone

In [33]:
    # "weight_actions": torch.zeros((hyper_rollout_steps, weight_action_dim)).to(device),
    #             "logprobs": torch.zeros((hyper_rollout_steps, 1)).to(device),
    #             "dones": torch.zeros((hyper_rollout_steps, 1)).to(device),
    #             "values"

In [36]:
# hyperagent
from modules.hyper_agent import HyperAgent
device = "cuda:1"
hyperagent = HyperAgent(cfg).to(device)

# hyper stroage
obs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_observation_space.shape).to(device)
actions = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_action_space.shape).to(device)
logprobs = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
rewards = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
dones = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
values = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
storage = TensorDict({
            "obs": obs,
            "actions": actions,
            "rewards": rewards,
            "dones": dones,
            }, batch_size=[exp_cfg.num_rollout_steps, exp_cfg.num_envs])

hyper_rollout_steps = 8
# 17 + 64 + 64 + 6 + 64 + 64 + 6
obs_dim = np.prod(env.observation_space.shape)
mid_dim = 64
act_dim = np.prod(env.action_space.shape)
weight_action_dim = obs_dim + 4*mid_dim + 2*act_dim
hyper_storage = TensorDict({
                "trajectories": torch.stack([storage.clone()for i in range(hyper_rollout_steps)], dim=0),
                "weight_actions": torch.zeros((hyper_rollout_steps, weight_action_dim)).to(device),
                "logprobs": torch.zeros((hyper_rollout_steps, 1)).to(device),
                "rewards": torch.zeros((hyper_rollout_steps, 1)).to(device),
                "dones": torch.zeros((hyper_rollout_steps, 1)).to(device),
                "values": torch.zeros((hyper_rollout_steps, 1)).to(device),
                }, batch_size=[hyper_rollout_steps,])

# env reset
next_obs = env.reset()
next_obs = torch.Tensor(next_obs).to(device)
hyperdone = torch.zeros(exp_cfg.num_envs).to(device)
print(next_obs.shape)
print(hyperdone.shape)
sampled_actions = np.array([env.action_space.sample() for i in range(next_obs.shape[0])])
print(actions.shape)
action = torch.zeros_like(torch.tensor(sampled_actions))
reward = torch.zeros([action.shape[0], ])
print(rewards.shape)
obs_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_observation_space.shape).to(device)
actions_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs) \
    + env.single_action_space.shape).to(device)
rewards_ary = torch.zeros((exp_cfg.num_rollout_steps, exp_cfg.num_envs)).to(device)
obs_ary[0] = next_obs
actions_ary[0] = action
rewards_ary[0] = reward
trajectories = TensorDict({
            "obs": obs_ary,
            "actions": actions_ary,
            "rewards": rewards_ary,
            }, batch_size=[exp_cfg.num_rollout_steps, exp_cfg.num_envs])
global_step = 0

for update_idx in range(start_update_idx, 2):
    envs_returns = dict()
    envs_lengths = dict()
    

    for i in range(hyper_rollout_steps):
        # next_traj, hyper_env.step(weight_action)
        hyper_storage["trajectories"][i] = trajectories
        hyper_storage["dones"][i] = hyperdone
        with torch.no_grad():
            weight_action, log_prob, entropy, value, policy = hyperagent.get_action_value_policy(
                                                                trajectories,
                                                                env,
                                                                64,
                                                                None
                                                                )

        hyper_storage["values"][i] = value.flatten()
        hyper_storage["weight_actions"][i] = weight_action
        hyper_storage["logprobs"][i] = log_prob
        
        
        trajectories, reward, hyperdone = rollout(i, env, policy, hyper_storage,
                next_obs, hyperdone, envs_returns, envs_lengths, global_step)
        hyper_storage["rewards"][i] = reward
        # print(f"reward: {reward}")
    
    with torch.no_grad():
        next_value = hyperagent.get_value(trajectories).reshape(1, -1)
        hyper_storage['advantages'] = torch.zeros_like(hyper_storage['rewards']).to(device)
        lastgaelam = 0
        for t in reversed(range(hyper_rollout_steps)):
            if t == hyper_rollout_steps - 1:
                nextnonterminal = 1.0 - hyperdone
                nextvalues = next_value
            else:
                nextnonterminal = 1.0 - hyper_storage["dones"][t + 1]
                nextvalues =  hyper_storage['values'][t + 1]
            delta = hyper_storage['rewards'][t] + cfg.ppo.gamma * nextvalues * nextnonterminal - hyper_storage['values'][t]
            hyper_storage['advantages'][t] = lastgaelam = delta + cfg.ppo.gamma * cfg.ppo.gae_lambda * nextnonterminal * lastgaelam
        hyper_storage['returns'] = hyper_storage['advantages'] + hyper_storage['values']
    
    # hyper_storage["trajectories"].reshape(hyper_rollout_steps, exp_cfg.num_rollout_steps*exp_cfg.num_envs)
    batch_size = hyper_rollout_steps
    # num_minibatches = 4
    # minibatch_size = int(batch_size // num_minibatches)
    b_inds = np.arange(batch_size)
    for epoch in range(cfg.ppo.update_epochs):
        np.random.shuffle(b_inds)
        for mb_index in b_inds:
            mb_traj = hyper_storage["trajectories"][mb_index]
            mb_weight_action = hyper_storage["weight_actions"][mb_index]
            mb_log_prob = hyper_storage["logprobs"][mb_index]
            mb_advantages = hyper_storage["advantages"][mb_index]
            mb_returns = hyper_storage["returns"][mb_index]
            
            weight_action, new_logprob, entropy, new_value, policy = hyperagent.get_action_value_policy(
                                                                mb_traj,
                                                                env,
                                                                64,
                                                                mb_weight_action
                                                                )
            logratio = new_logprob.view(-1) - mb_log_prob.view(-1)
            ratio = logratio.exp()
            pg_loss1 = -mb_advantages * ratio
            pg_loss2 = -mb_advantages * torch.clamp(ratio, 1 - cfg.ppo.clip_coef, 1 + cfg.ppo.clip_coef)
            pg_loss = torch.max(pg_loss1, pg_loss2).mean()
            
            new_value = new_value.view(-1)
            v_loss = 0.5 * ((new_value - mb_returns) ** 2).mean()
            entropy_loss = entropy.mean()
            loss = pg_loss - cfg.ppo.ent_coef * entropy_loss + v_loss * cfg.ppo.vf_coef
            hyperagent.optim_zero_grad()
            loss.backward()
            hyperagent.optim_step()
        

kwargs: {}
torch.Size([1, 17])
torch.Size([1])
torch.Size([128, 1, 6])
torch.Size([128, 1])
global_step: 104. episodic_return=-116.12046813964844
global_step: 104.  episodic_length=1000


IndexError: too many indices for tensor of dimension 1

In [39]:
hyper_storage["trajectories"][0:2]

LazyStackedTensorDict(
    fields={
        actions: Tensor(shape=torch.Size([2, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([2, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([2, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([2, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([2, 128, 1]),
    device=None,
    is_shared=False)

In [43]:
b_inds = np.arange(8)
mb_ind = b_inds[0:2]


In [44]:
mb_ind.size

2

In [45]:
len(mb_ind)

2

In [53]:
hyper_storage["trajectories"]

LazyStackedTensorDict(
    fields={
        actions: Tensor(shape=torch.Size([8, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([8, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([8, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([8, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([8, 128, 1]),
    device=None,
    is_shared=False)

In [55]:
b_inds

array([2, 1, 6, 5, 0, 4, 7, 3])

In [ ]:
np.random.shuffle(b_inds)

In [57]:
hyper_storage["trajectories"].batch_size

torch.Size([8, 128, 1])

In [67]:
index = torch.randint(8, (2,)).to(device)
index

tensor([4, 0], device='cuda:1')

In [75]:
id_ten = torch.tensor(index)
len(id_ten)

/tmp/ipykernel_2397141/1910030952.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  id_ten = torch.tensor(index)


2

In [71]:
b_inds = np.arange(8)
np.random.shuffle(b_inds)
mb_inds = b_inds[0:2]
mb_inds

array([3, 2])

In [72]:
gathered_tensordict = hyper_storage.gather(dim=0, index=mb_inds)


TypeError: gather() received an invalid combination of arguments - got (TensorDict, int, numpy.ndarray, out=NoneType), but expected one of:
 * (Tensor input, int dim, Tensor index, *, bool sparse_grad, Tensor out)
 * (Tensor input, name dim, Tensor index, *, bool sparse_grad, Tensor out)


In [64]:
import tensordict

In [68]:
gathered_tensordict = hyper_storage.gather(dim=0, index=index)

In [69]:
gathered_tensordict

TensorDict(
    fields={
        advantages: Tensor(shape=torch.Size([2, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([2, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        logprobs: Tensor(shape=torch.Size([2, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        returns: Tensor(shape=torch.Size([2, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([2, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        trajectories: TensorDict(
            fields={
                actions: Tensor(shape=torch.Size([2, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
                dones: Tensor(shape=torch.Size([2, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
                obs: Tensor(shape=torch.Size([2, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
                rewards: Tensor(shape=torch.Size([2, 1

In [56]:
hyper_storage["trajectories"].permute(b_inds)

RuntimeError: number of dims don't match in permute (got 8, expected 3

In [52]:
indices = np.array((0, 1))
hyper_storage["trajectories"][0:2].reshape(128, 2)

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([128, 2, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([128, 2]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([128, 2, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([128, 2]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([128, 2]),
    device=None,
    is_shared=False)

In [37]:
trajectories

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([128, 1]),
    device=None,
    is_shared=False)

In [ ]:
hyper_storage["trajectories"]["dones"][0:1]

tensor([[[0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [0.],
         [

In [ ]:
torch.logical_or

In [29]:
a = torch.tensor(int(False))
a

tensor(0)

In [26]:
hyperdone

tensor(False, device='cuda:1')

In [128]:
hyper_storage["logprobs"][0].view(-1)

tensor([0.], device='cuda:1')

In [125]:
hyper_storage["trajectories"][0]

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([128, 1]),
    device=None,
    is_shared=False)

In [124]:
hyper_storage["weight_actions"].shape

torch.Size([4, 285])

In [115]:
hyper_storage

TensorDict(
    fields={
        dones: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        logprobs: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        trajectories: LazyStackedTensorDict(
            fields={
                actions: Tensor(shape=torch.Size([4, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
                dones: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
                obs: Tensor(shape=torch.Size([4, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
                rewards: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
            batch_size=torch.Size([4, 128, 1]),
            device=None,
            is_shared=False),
        values: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        weight_actions: Tensor(sh

In [113]:
hyper_storage["trajectories"]

LazyStackedTensorDict(
    fields={
        actions: Tensor(shape=torch.Size([4, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([4, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([4, 128, 1]),
    device=None,
    is_shared=False)

In [114]:
hyper_storage["trajectories"].reshape(hyper_rollout_steps, exp_cfg.num_rollout_steps)

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([4, 128, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([4, 128]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([4, 128, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([4, 128]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([4, 128]),
    device=None,
    is_shared=False)

In [108]:
hyper_storage["trajectories"][0:1].reshape(-1)

TensorDict(
    fields={
        actions: Tensor(shape=torch.Size([128, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
        dones: Tensor(shape=torch.Size([128]), device=cuda:1, dtype=torch.float32, is_shared=True),
        obs: Tensor(shape=torch.Size([128, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
        rewards: Tensor(shape=torch.Size([128]), device=cuda:1, dtype=torch.float32, is_shared=True)},
    batch_size=torch.Size([128]),
    device=None,
    is_shared=False)

In [104]:
hyper_storage

TensorDict(
    fields={
        dones: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        logprobs: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        trajectories: LazyStackedTensorDict(
            fields={
                actions: Tensor(shape=torch.Size([4, 128, 1, 6]), device=cuda:1, dtype=torch.float32, is_shared=True),
                dones: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
                obs: Tensor(shape=torch.Size([4, 128, 1, 17]), device=cuda:1, dtype=torch.float32, is_shared=True),
                rewards: Tensor(shape=torch.Size([4, 128, 1]), device=cuda:1, dtype=torch.float32, is_shared=True)},
            batch_size=torch.Size([4, 128, 1]),
            device=None,
            is_shared=False),
        values: Tensor(shape=torch.Size([4, 1]), device=cuda:1, dtype=torch.float32, is_shared=True),
        weight_actions: Tensor(sh

tensor([[[-0.7030,  0.9323,  0.7391,  0.7484, -0.8807, -0.1586, -1.2903,
          -0.7783, -1.4226, -0.2737,  1.2954, -1.6936,  0.2868,  0.5331,
          -1.2357,  1.4036,  0.1558]],

        [[-0.7530,  0.7181, -0.5714, -0.6478, -0.8127,  0.6499,  0.8299,
           0.7254,  0.2494,  0.4721,  0.0887, -0.7353, -0.7354,  0.6125,
           1.1471,  0.6737,  0.6369]]], device='cuda:1')